#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

# query1: Query artist, song, and length of song based on session_id and item_in_session. Partition key is seeesin_id and clustering key is item_in_session

### Create tables

In [8]:
query1 = "SELECT artist, song, length FROM song_details WHERE session_id = 338 AND item_in_session = 4"

session.execute("""CREATE TABLE IF NOT EXISTS song_details (
session_id INT,
item_in_session INT,
artist VARCHAR,
length FLOAT,
song VARCHAR,
PRIMARY KEY (session_id, item_in_session))""")
                    

### Insert into data

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details (session_id, item_in_session, artist, length, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

### CQL query

In [14]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print("artist name: ", row.artist, "| song title: ", row.song, "| song length: ", row.length)

artist name:  Faithless | song title:  Music Matters (Mark Knight Dub) | song length:  495.30731201171875


# query2: Query artist, song, first_name and last_name based on user_id and session_id. Partition keys are user_id and session_id. Clustering key is item_in_session.

### Create table, insert data and CQL query

In [15]:
query2 = "SELECT artist, song, first_name, last_name FROM song_playlist_session WHERE user_id = 10 AND session_id = 182"

session.execute("""CREATE TABLE IF NOT EXISTS song_playlist_session (
user_id INT,
session_id INT,
item_in_session INT,
artist VARCHAR,
first_name VARCHAR,
last_name VARCHAR,
song VARCHAR,
PRIMARY KEY ((user_id, session_id), item_in_session))""")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (user_id, session_id, item_in_session, artist, first_name, last_name, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

try:
    rows = session.execute(query2)
except Exception as e:
    pritn(e)

for row in rows:
    print("artist name: ", row.artist, "| song title: ", row.song, "| first name: ", row.first_name, "| last name: " row.last_name)


SyntaxError: invalid syntax (<ipython-input-15-21449518431c>, line 29)

# query3: Query first_name and last_name based on song. Partitoin key is song. Clustering key is user_id.

### Create table, insert data and CQL query

In [12]:
query3 = "SELECT first_name, last_name FROM users_listen_song WHERE song = 'All Hands Against His Own'"

session.execute("""CREATE TABLE IF NOT EXISTS users_listen_song (
song VARCHAR,
user_id INT,
first_name VARCHAR,
last_name VARCHAR,
PRIMARY KEY (song, user_id)) """)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_listen_song (song, user_id, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( line[9], int(line[10]), line[1], line[4]))

try:
    rows = session.execute(query3)
except Exception as e:
    prtin(e)

for row in rows:
    print(row.first_name, row.last_name)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS song_playlist_session")
session.execute("DROP TABLE IF EXISTS users_listen_song")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()